In [0]:
                                          ## Initialize
!pip install geopandas

import geopandas as gpd
import numpy as np
import pandas as pd
import time

from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

    100% |████████████████████████████████| 931kB 21.1MB/s 
    100% |████████████████████████████████| 10.8MB 2.7MB/s 
    100% |████████████████████████████████| 17.9MB 1.9MB/s 
  Stored in directory: /root/.cache/pip/wheels/db/bf/bc/06a3e1bfe0ab27d2e720ceb3cff3159398d92644c0cec2c125
Successfully built munch


In [0]:
## Load data
## https://data.colorado.gov/Demographics/Census-Blocks-in-Colorado-2010/xipb-k5bu
df_cbg = gpd.read_file('Census Block Groups in Colorado 2010.geojson')
print(df_cbg.tail())

     pct_amind      intptlon households med_age occupied v_s_notocc white_nh  \
3527       0.4  -104.8419816      779.0    49.7    779.0        0.0   1169.0   
3528      0.87  -104.7994864      613.0    34.0    613.0        0.0   1026.0   
3529      0.43  -104.7965144      582.0    34.4    582.0        0.0    913.0   
3530      0.59  -104.7962523      433.0    33.0    433.0        0.0    806.0   
3531       0.7  -104.6895702      617.0    30.4    617.0        3.0   1224.0   

     female state gq_o_noni  \
3527  938.0    08       0.0   
3528  907.0    08       0.0   
3529  822.0    08       0.0   
3530  659.0    08       0.0   
3531  922.0    08       0.0   

                            ...                         gq_nurs  owner avg_hh  \
3527                        ...                             0.0  593.0   2.25   
3528                        ...                             0.0  486.0    3.0   
3529                        ...                             0.0  446.0   2.77   
3530    

In [0]:
## https://data.colorado.gov/Transportation/Counties-in-Colorado/67vn-ijga
df_county =pd.read_csv('COUNTIES.csv')
df_county_clean = df_county[['COUNTY','CO_FIPS']]
## https://s3.amazonaws.com/data.openaddresses.io/runs/608168/us/co/statewide.zip
df_add = pd.read_csv('DenverAddress.csv')
print(df_add.shape,'\n',df_add.head())

(318102, 11) 
           LON        LAT NUMBER               STREET      UNIT  CITY  \
0 -104.881916  39.796684   5450  N Central Park Blvd       NaN   NaN   
1 -104.922850  39.678364   5570          E Evans Ave       NaN   NaN   
2 -104.883115  39.797146   5500  N Central Park Blvd       NaN   NaN   
3 -104.952877  39.719160   2930            E 2nd Ave  Unit 611   NaN   
4 -104.998813  39.717723    900            W 1st Ave       NaN   NaN   

   DISTRICT  REGION  POSTCODE  ID              HASH  
0       NaN     NaN       NaN NaN  a94f243615dc9905  
1       NaN     NaN       NaN NaN  f18e96dd5bf8e4d3  
2       NaN     NaN       NaN NaN  22efb5ebf257feeb  
3       NaN     NaN       NaN NaN  197894e3f12257a1  
4       NaN     NaN       NaN NaN  bf04798f244b1d24  


In [0]:
#Format new address columns
df_add['Address'] = df_add['NUMBER'].astype('str')+ ' '+ df_add['STREET']
df_add['Address'] = df_add['Address'].str.upper()
df_add_clean = df_add.drop(columns=['NUMBER','STREET', 'UNIT', 'CITY', 'DISTRICT','REGION','POSTCODE','ID','HASH'])
df_add_clean=df_add_clean.rename(index=str, columns={'LON':'Longitude','LAT':'Latitude'})
df_add_clean = df_add_clean.drop_duplicates(subset='Address',keep='first')
df_add_clean = df_add_clean.dropna()

df_add_clean['Tract']=''
df_add_clean['Tract-CensusBlockGroup']=''
df_add_clean.head()

In [0]:
#Format new CensusBlock columns
df_cbg2 = df_cbg[['county','tract','blkgrp','geometry']]
df_cbg2['county'] = df_cbg2['county'].astype(int)
df_cbg_clean = df_cbg2.merge(df_county_clean, left_on = "county", right_on = 'CO_FIPS')
df_cbg_clean = df_cbg_clean.rename(index=str, columns={'COUNTY':'CountyName','tract':'Tract','geometry':'Geometry','blkgrp':'CensusBlockGroup'})
df_cbg_clean = df_cbg_clean[df_cbg_clean['CountyName']=='DENVER'] #CO_FIPS==31
df_cbg_clean = df_cbg_clean.drop(columns=['county','CO_FIPS','CountyName'])
df_cbg_clean['Tract-CensusBlockGroup'] = df_cbg_clean['Tract'].astype(str)+'-'+df_cbg_clean['CensusBlockGroup'].astype(str)
df_cbg_clean.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Tract,CensusBlockGroup,Geometry,Tract-CensusBlockGroup
1812,003601,4,(POLYGON ((-104.9634390000001 39.7657500000000...,003601-4
1813,002702,4,(POLYGON ((-104.9867450000001 39.7320400000000...,002702-4
1814,015600,3,(POLYGON ((-105.0250510000001 39.6749540000000...,015600-3
1815,002701,4,(POLYGON ((-104.9823000000001 39.7351520000000...,002701-4
1816,003601,5,(POLYGON ((-104.9684250000001 39.7657570000000...,003601-5


In [0]:
#Check shape and NaN
def nancheck(df):
    print('Shape of the dataframe:',df.shape,'\n')
    print('Missing value counts:')
    print(df.shape[0]-df.count(),'\n') 
    return

nancheck(df_cbg_clean)
nancheck(df_add_clean)

Shape of the dataframe: (481, 4) 

Missing value counts:
Tract                     0
CensusBlockGroup          0
Geometry                  0
Tract-CensusBlockGroup    0
dtype: int64 

Shape of the dataframe: (203112, 7) 

Missing value counts:
Longitude            0
Latitude             0
Address              0
Tract                0
CensusBlock          0
County               0
Tract-CensusBlock    0
dtype: int64 



In [0]:
for i in range(len(df_add_clean)):
    point=Point(df_add_clean['Longitude'][i],df_add_clean['Latitude'][i])
    for j in range(len(df_cbg_clean)): 
        if df_cbg_clean['Geometry'][j].contains(point):
            df_add_clean['Tract'][i]= df_cbg_clean['Tract'][j]
            df_add_clean['Tract-CensusBlockGroup'][i]= df_cbg_clean['Tract-CensusBlockGroup'][j]
            break
    if (i%1000==0): print(i,df_add_clean.iloc[i])   

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


0 Longitude                                 -104.882
Latitude                                   39.7967
Address                   5450 N CENTRAL PARK BLVD
Tract                                       004106
CensusBlock                                       
County                                            
Tract-CensusBlock                                 
Tract-CensusBlockGroup                    004106-1
Name: 0, dtype: object
1000 Longitude                        -104.893
Latitude                          39.7595
Address                   2979 N UINTA ST
Tract                              004107
CensusBlock                              
County                                   
Tract-CensusBlock                        
Tract-CensusBlockGroup           004107-1
Name: 3178, dtype: object
2000 Longitude                        -105.039
Latitude                          39.7364
Address                   1290 N PERRY ST
Tract                              000702
CensusBlock                

In [0]:
df_add_clean['geonum']='108031'+df_add_clean['Tract']+df_add_clean['CensusBlockGroup']
df_add_clean.tail()

In [0]:
#Merger the two
df = df1.merge(df2, left_on = "Address", right_on = 'Address')

In [0]:
#Clean the new dataframe to remove unneeded columns and rename the rest
df = df.drop(columns=['index_x','index_y', 'UNIT', 'CITY', 'DISTRICT','REGION','POSTCODE','ID','HASH', 'NUMBER' ,'STREET','RES_STATE','RTD','BALLOT_REQUEST_DATE',''])
df=df.rename(index=str, columns={'VOTER_ID':'VoterID','VOTER_NAME':'VoterName','BIRTH_YEAR':'BirthYear','RES_ADDRESS':'FullAddress','RES_CITY':'City','RES_ZIP_CODE':'ZIP','PARTY':'Party','PRECINCT':'Precinct','COUNCIL_DISTRICT':'CouncilDistrict','SCHOOL_DISTRICT':'SchoolDistrict','STATE_HOUSE':'StateHouse','STATE_SENATE':'StateSenate','CONGRESSIONAL_DISTRICT':'CongressionalDistrict'
df.head(20)

,VOTER_ID,VOTER_NAME,BIRTH_YEAR,RES_ADDRESS,RES_CITY,RES_STATE,RES_ZIP_CODE,PARTY,PRECINCT,COUNCIL_DISTRICT,...,BALLOT_RETURNED_DATE,BALLOT_STAGE,BALLOT_STATUS,BALLOT_STATUS_REASON,IN_PERSON_VOTE,IN_PERSON_VOTE_DATE,VOTED_PARTY,Address,LON,LAT
0,601121850,OWEN ALEXANDER MCVITTY,1995,1 E BAYAUD AVE APT 2,DENVER,CO,80209,UAF,1320216218,7,...,11/6/2018,Validated,Accepted,NaN,No,NaN,NaN,1 E BAYAUD AVE,-104.987145,39.714908
1,601844890,MEGAN ANN RITTER,1993,1 E BAYAUD AVE APT 10,DENVER,CO,80209,DEM,1320216218,7,...,NaN,Sent,NaN,NaN,No,NaN,NaN,1 E BAYAUD AVE,-104.987145,39.714908
2,4296150,BRIAN QUINCY BUCCIARELLI,1970,1 E BAYAUD AVE UNIT 5,DENVER,CO,80209,DEM,1320216218,7,...,NaN,Sent,NaN,NaN,No,NaN,NaN,1 E BAYAUD AVE,-104.987145,39.714908
3,601788146,JASON ANDREW MURPHY,1993,1 E BAYAUD AVE APT 3,DENVER,CO,80209,UAF,1320216218,7,...,11/6/2018,Validated,Accepted,NaN,No,NaN,NaN,1 E BAYAUD AVE,-104.987145,39.714908
4,200129262,DUANE LEE CRANMER,1973,1 E BAYAUD AVE UNIT 8,DENVER,CO,80209,DEM,1320216218,7,...,NaN,Undeliverable,NaN,NaN,No,NaN,NaN,1 E BAYAUD AVE,-104.987145,39.714908
5,601156167,ELIZABETH TWINING HEARN,1992,1 E BAYAUD AVE APT 1,DENVER,CO,80209,UAF,1320216218,7,...,NaN,Sent,NaN,NaN,No,NaN,NaN,1 E BAYAUD AVE,-104.987145,39.714908
6,2962344,ADAM BENJAMIN GILDAR,1984,1 E BAYAUD AVE APT 9,DENVER,CO,80209,DEM,1320216218,7,...,10/31/2018,Validated,Rejected,No Signature,No,NaN,NaN,1 E BAYAUD AVE,-104.987145,39.714908
7,600239323,CHRISTOPHER SHAWN CARLTON JR,1991,1 E BAYAUD AVE UNIT 4,DENVER,CO,80209,DEM,1320216218,7,...,11/4/2018,Validated,Accepted,NaN,No,NaN,NaN,1 E BAYAUD AVE,-104.987145,39.714908
8,2641930,ANTOINETT E FREED,1950,1 N ASH ST,DENVER,CO,80220,DEM,1310616622,5,...,11/2/2018,Validated,Accepted,NaN,No,NaN,NaN,1 N ASH ST,-104.938746,39.716458
9,5720149,KATIE LYNN ALBRIGHT,1981,1 N BELLAIRE ST,DENVER,CO,80220,DEM,1310616622,5,...,11/6/2018,Validated,Accepted,NaN,No,NaN,NaN,1 N BELLAIRE ST,-104.937644,39.716428


In [0]:
# This is a list of Voter_IDs that do not have lat long corrdinates
dif = pd.concat([df1['VOTER_ID'],df['VOTER_ID']]).drop_duplicates(keep=False)
dif

11          4902339
13          2869505
14          3998589
15        601285635
19        601263590
20        200032628
31          2429258
32          2404848
34        600295244
35        601483330
37        601784840
38        601737334
39        600302141
40        200380178
86          2893472
91          2672521
92          2431101
146       601732914
167         2701965
169         2697816
172         2609315
173       600499541
174         6948076
175         2481990
177       601249317
178          947651
179         2646439
192       200229442
320       600476335
433       601374155
            ...    
445735      2543958
445848      2832190
445904    600526742
446024      3972105
446025    601940843
446026      2952712
446027      2450948
446028      2889081
446031      2473278
446032      2955842
446033      2925232
446034    600185812
446225    200361399
446226       673418
446231    601299468
446234    601887925
446237    600610415
446238       662840
446239    601845054


In [0]:
# Checking the shapes of the dataframes
print(df1.shape)
print(df.shape)

(446258, 25)
(415243, 26)


In [0]:
#Making sure there are no NAN errors
print(df['LAT'].isnull().values.any())
print(df['LON'].isnull().values.any())

False
False


In [0]:
df['LON'].isnull().values.any()

False

In [0]:
# Saving the new merged file/checkpoint
def write_checkpoint(df, dfname):
  filename=time.strftime("%Y%m%d-%H%M")+' '+ dfname+'.csv'
  df.to_csv(filename, index=False)
  return

def read_checkpoint(filename):
    return pd.read_csv(filename,index_col=0)
  
write_checkpoint(df_add_clean,'DenverAddressCensusBlockGroup')